# Processing Big Data - Deequ Analysis

© Explore Data Science Academy

## Honour Code
I **Daluxolo**, **Mbatha**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.


## Context

Having completed manual data quality checks, it should be obvious that the process can become quite cumbersome. As the Data Engineer in the team, you have researched some tools that could potentially save the team from having to do this cumbersome work. In your research, you have come a across a tool called [Deequ](https://github.com/awslabs/deequ), which is a library for measuring the data quality of large datasets.

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://github.com/Explore-AI/Pictures/raw/master/data_engineering/transform/predict/DataQuality.jpg"
     alt="Data Quality"
     style="float: center; padding-bottom=0.5em"
     width=100%/>
     <p><em>Figure 1. Six dimensions of data quality</em></p>
</div>

You present this tool to your manager; he is quite impressed and gives you the go-ahead to use this in your implementation. You are now required to perform some data quality tests using this automated data testing tool.
 

> ## 🚩️ Important Notice 🚩️
>
>To successfully run `pydeequ` without any errors, please make sure that you have an environment that is running pyspark version 3.0.
> You are advised to **create a new conda environment** and install this specific version of pyspark to avoid any technical issues:
>
> `pip install pyspark==3.0`

<br>

## Import dependencies

If you do not have `pydeequ` already installed, install it using the following command:
- `pip install pydeequ`

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydeequ
from pydeequ.analyzers import *
from pydeequ.profiles import *
from pydeequ.suggestions import *
from pydeequ.checks import *
from pydeequ.verification import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType, DoubleType, IntegerType, DateType, NumericType, StructType, StringType, StructField

In [3]:
spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

## Read data into spark dataframe

In this notebook, we set out to run some data quality tests, with the possiblity of running end to end on the years 1963, 1974, 1985, 1996, 2007, and 2018. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Data_ingestion_student_version.ipynb` notebook to create the parquet files for the following years:
>       - 1963
>       - 1974
>       - 1985
>       - 1996
>       - 2007
>       - 2018
>
>2. Ingest the data for the for the years given above. You should only do it one year at a time.
>3. Ingest the metadata file.


When developing your code, it will be sufficient to focus on a single year. However, after your development is done, you will need to run this notebook for all of the given years above so that you can answer all the questions given in the Data Testing MCQ.

In [39]:
#TODO: Write your code here
# Use this variable (year) to determine which year your are focusing on
year = 1985
df = spark.read.parquet('/Users/daluxolombatha/Desktop/three/1985/*')
df.show(3)

+----------+---------+---------+---------+---------+-------------+--------+-----+
|      date|     open|     high|      low|    close|    adj_close|  volume|stock|
+----------+---------+---------+---------+---------+-------------+--------+-----+
|1985-12-19|11.858805|   12.015|11.834775|11.834775|     6.174869|856500.0|   AA|
|1985-12-19|      0.0|     12.4|     12.2|     12.4|   10.7758875|  4000.0| AAME|
|1985-12-19|1.2037038|1.2037038|1.1666666|1.2037038|-1.5022586E21|665500.0|  AAN|
+----------+---------+---------+---------+---------+-------------+--------+-----+
only showing top 3 rows



In [40]:
float(df.describe("close").filter("summary = 'max'").select("close").first().asDict()['close'])

221250.0

In [41]:
df.createOrReplaceTempView('st')
spark.sql("""SELECT * 
        FROM st
        WHERE adj_close < 0
        """).show(2)

+----------+---------+---------+---------+---------+-------------+--------+-----+
|      date|     open|     high|      low|    close|    adj_close|  volume|stock|
+----------+---------+---------+---------+---------+-------------+--------+-----+
|1985-12-19|1.2037038|1.2037038|1.1666666|1.2037038|-1.5022586E21|665500.0|  AAN|
|1985-12-19|      0.0| 2.234375|   2.1875| 2.203125|-0.0034884836|243200.0|  CLF|
+----------+---------+---------+---------+---------+-------------+--------+-----+
only showing top 2 rows



## **Run tests on the dataset**

## Test 1 - Null values ⛔️
For the first test, you are required to check the data for completeness.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for missing values in the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*


In [42]:
#TODO: Write your code here
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("date")) \
                    .addAnalyzer(Completeness("open")) \
                    .addAnalyzer(Completeness("low")) \
                    .addAnalyzer(Completeness("high")) \
                    .addAnalyzer(Completeness("close")) \
                    .addAnalyzer(Completeness("adj close")) \
                    .addAnalyzer(Completeness("volume")) \
                    .addAnalyzer(Completeness("stock")) \
                .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+--------+------------+------------------+
| entity|instance|        name|             value|
+-------+--------+------------+------------------+
| Column|    date|Completeness|               1.0|
| Column|   close|Completeness|0.9999595333618527|
| Column|     low|Completeness|0.9999595333618527|
| Column|    high|Completeness|0.9999595333618527|
|Dataset|       *|        Size|          172982.0|
| Column|   stock|Completeness|               1.0|
| Column|  volume|Completeness|0.9999595333618527|
| Column|    open|Completeness|0.9999595333618527|
+-------+--------+------------+------------------+



## Test 2 - Zero Values 🅾️

For the second test, you are required to check for zero values within the dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for zero values within the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [43]:
#TODO: Write your code here
check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 100000) \
        .hasMin("open", lambda x: x == 0) \
        .hasMin("high", lambda x: x == 0) \
        .hasMin("low", lambda x: x == 0) \
        .hasMin("close", lambda x: x == 0) \
        .hasMin("adj close", lambda x: x == 0) \
        .hasMin("volume", lambda x: x == 0)) \
    .run()
    
checkResult_df1 = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df1.show()

+------------+-----------+------------+--------------------+-----------------+--------------------+
|       check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+------------+-----------+------------+--------------------+-----------------+--------------------+
|Review Check|    Warning|     Warning|SizeConstraint(Si...|          Success|                    |
|Review Check|    Warning|     Warning|MinimumConstraint...|          Success|                    |
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Value: 0.00455729...|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Value: 0.00325520...|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Value: 0.00325520...|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Input data does n...|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Success|                    |


## Test 3 - Negative values ➖️
The third test requires you to check that all values in the data are positive.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check negative values within the dataset. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [17]:
#TODO: Write your code here

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 100000) \
        .hasMin("open", lambda x: x < 0) \
        .hasMin("high", lambda x: x < 0) \
        .hasMin("low", lambda x: x < 0) \
        .hasMin("close", lambda x: x < 0) \
        .hasMin("adj close", lambda x: x < 0) \
        .hasMin("volume", lambda x: x < 0)) \
    .run()

checkResult_df2 = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df2.show()

+------------+-----------+------------+--------------------+-----------------+--------------------+
|       check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+------------+-----------+------------+--------------------+-----------------+--------------------+
|Review Check|    Warning|     Warning|SizeConstraint(Si...|          Success|                    |
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Value: 0.0 does n...|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Value: 0.00455729...|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Value: 0.00325520...|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Value: 0.00325520...|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Input data does n...|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Value: 0.0 does n...|


## Test 4 - Determine Maximum Values ⚠️

For the fourth test, we want to find the maximum values in the dataset for the numerical fields. Extremum values can often be used to define an upper bound for the column values so we can define them as the threshold values. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Column Profiler Runner` to generate summary statistics for all the available columns. 
>2. Extract the maximum values for all the numeric columns in the data.
>
> *You may use as many cells as necessary*

In [18]:
#TODO: Write your code here
print("Test 4")
results=ColumnProfilerRunner(spark).onData(df).run()
for col, profile in results.profiles.items():
    print(profile)

Test 4
StandardProfiles for column: stock: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 674,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 172982
    },
    "histogram": null
}
NumericProfiles for column: open: {
    "completeness": 0.9999595333618527,
    "approximateNumDistinctValues": 11783,
    "dataType": "Fractional",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": null,
    "kll": "None",
    "mean": 105.63464648399778,
    "maximum": 100000.0,
    "minimum": 0.0,
    "sum": 18272152.975569516,
    "stdDev": 2630.228499061416,
    "approxPercentiles": []
}
NumericProfiles for column: low: {
    "completeness": 0.9999595333618527,
    "approximateNumDistinctValues": 18488,
    "dataType": "Fractional",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": null,
    "kll": "None",


## Test 5 - Stock Tickers 💹️

For the fifth test, we want to determine if the stock tickers contained in our dataset are consistent. To do this, you will need to make use of use of the metadata file to check that the stock names used in the dataframe are valid. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine if the stock tickers contained in the dataset appear in the metadata file.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [20]:
#TODO: Write your code here

## Test 6 - Duplication 👥️
Lastly, we want to determine the uniqueness of the items found in the dataframe. You need to make use of the Verification Suite to check for the validity of the stock tickers. 

Similar to the previous notebook - `Data_profiling_student_version.ipynb`, the first thing to check will be if the primary key values within the dataset are unique - in our case, that will be a combination of the stock name and the date. Secondly, we want to check if the entries are all unique, which is done by checking for duplicates across that whole dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine the uniqueness of entries contained within the dataset.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*



In [21]:
#TODO: Write your code here